In [1]:
#1)Importar librerías de python para análisis de datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#2)Crear un dataframe y copiar la base de datos
df = pd.read_csv( "titanic.csv" )


In [3]:
#3)Mostrar el dataframe
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
# exploramos las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Preparar los datos

In [5]:
# Eliminanos las columnas que no aportan a la predicción
df.drop( [ "PassengerId", "Name", "Ticket", "Cabin" ], axis = 1, inplace = True )
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [14]:
# Identificar valores nulos (NaN)
df.isnull().sum()

promedio_edad = df[ "Age" ].mean()
moda_embarked = df[ "Embarked" ].mode() [0]

print( type(promedio_edad), promedio_edad )
print( type(moda_embarked ), moda_embarked)

<class 'numpy.float64'> 29.69911764705882
<class 'str'> S


In [16]:
# Rellenamos los valores nulos
df[ "Age" ].fillna( promedio_edad, inplace = True )
df[ "Embarked" ].fillna( moda_embarked, inplace = True  )

df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [19]:
# Todas las columnas deben ser numéricas
print( df["Sex"].unique(), df["Embarked"].unique() )

df["Sex"].replace(  ['male' ,'female'], [0, 1], inplace = True  )
df["Embarked"].replace( ['S', 'C', 'Q'],  [0, 1, 2], inplace = True  )

print( df["Sex"].unique(), df["Embarked"].unique() )

['male' 'female'] ['S' 'C' 'Q']
[0 1] [0 1 2]


In [21]:
# Identificamos la variable dependiente "Y" y las variables independientes "X"
y = df[ "Survived" ]
x = df[ [ "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked" ] ]

print( "y", y.shape, "x", x.shape )

y (891,) x (891, 7)


# MACHINE LEARNING

In [22]:
# Aprendizaje SUPERVISADO -> conocemos la variable a predecir
# Importar los paquetes y funciones de Machine Learning
# importamos la función "train_test_split" dividir el Dataframe (x, y) de forma aleatoria en datos de entrenamiento y pruebas
from sklearn.model_selection import train_test_split

#Importamos los algoritmos de predicción

#Algoritmo 1: Bosque aleatorio y creamos el modelo con los siguientes parámetros 
#( max_depth = 5, n_estimators = 200, random_state = 1)
from sklearn.ensemble import RandomForestClassifier
modelo1 = RandomForestClassifier( max_depth = 5, n_estimators = 200, random_state = 1)

#Algoritmo 2: Regresión logística (random_state = 1)
from sklearn.linear_model import LogisticRegression
modelo2 = LogisticRegression( random_state = 1 )

#Algoritmo 3: Máquinas de soporte vectorial ( kernel = "rbf", random_state = 1 )
from sklearn.svm import SVC
modelo3 = SVC( kernel = "rbf", random_state = 1 )

#Algoritmo 4: Vecinos Cercanos ( n_neighbors = 3 )
from sklearn.neighbors import KNeighborsClassifier
modelo4 = KNeighborsClassifier( n_neighbors = 3 )

#Uso de métricas
from sklearn.metrics import accuracy_score

In [23]:
# Separamos los datos de entrenamiento y pruebas
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size = 0.20 )

print( x_train.shape, x_test.shape, y_train.shape, y_test.shape )

(712, 7) (179, 7) (712,) (179,)


In [24]:
# Algoritmo 1: Entrenamiento
modelo1.fit( x_train, y_train )

RandomForestClassifier(max_depth=5, n_estimators=200, random_state=1)

In [25]:
# Predicción de los datos de Test
y_pred = modelo1.predict( x_test )

In [30]:
# Mostrar la exactitud de la predicción
# Métrica de evaluación
# exactitud = (Predicciones_correctas / Numero_total_predicciones)
print( y_pred )
print()
print( y_test.to_numpy() )

exactitud_modelo1 = accuracy_score( y_test, y_pred )

print(exactitud_modelo1)

[0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]

[0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0
 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1]
0.8156424581005587


In [31]:
# Algoritmo 2: Entrenamiento
modelo2.fit( x_train, y_train )

LogisticRegression(random_state=1)

In [32]:
# Predicción de los datos de Test
y_pred = modelo2.predict( x_test )

print( y_pred )
print()
print( y_test.to_numpy() )

exactitud_modelo2 = accuracy_score( y_test, y_pred )
print()
print(exactitud_modelo2)

[0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0
 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0]

[0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0
 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1]

0.770949720670391


In [33]:
# Algoritmo 3: Entrenamiento
modelo3.fit( x_train, y_train )

SVC(random_state=1)

In [34]:
# Predicción de los datos de Test
y_pred = modelo3.predict( x_test )

print( y_pred )
print()
print( y_test.to_numpy() )

exactitud_modelo3 = accuracy_score( y_test, y_pred )
print()
print(exactitud_modelo3)

[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]

[0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0
 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1]

0.6536312849162011


In [35]:
# Algoritmo 4: Entrenamiento
modelo4.fit( x_train, y_train )

KNeighborsClassifier(n_neighbors=3)

In [36]:
# Predicción de los datos de Test
y_pred = modelo4.predict( x_test )

print( y_pred )
print()
print( y_test.to_numpy() )

exactitud_modelo4 = accuracy_score( y_test, y_pred )
print()
print(exactitud_modelo4)

[0 1 0 0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1
 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 0 1 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1
 0 0 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1]

[0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0
 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1]

0.6983240223463687


In [37]:
#Seleccionar el algoritmo a utilizar
print( "1. RandomForestClasiffier",  exactitud_modelo1 )
print( "2. LogisticRegression",  exactitud_modelo2 )
print( "3. SVC",  exactitud_modelo3 )
print( "4. KNeighborsClassifier",  exactitud_modelo4 )

1. RandomForestClasiffier 0.8156424581005587
2. LogisticRegression 0.770949720670391
3. SVC 0.6536312849162011
4. KNeighborsClassifier 0.6983240223463687


In [43]:
# Seleccionamos el modelo que realiza mejor las predicciones
# Seleccionamos el Modelo 1 : RandomForestClasiffier

# Un nuevo pasajero

df_nuevo_pasajero = pd.DataFrame( { "Pclass": [3],
                                   "Sex": [0],
                                   "Age": [2],
                                   "SibSp": [0],
                                   "Parch": [0],
                                   "Fare": [300],
                                   "Embarked":[2]
                                    } )

df_nuevo_pasajero

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,2,0,0,300,2


In [44]:
# Sobreviviría en el Titanic?
y_pred_yo = modelo1.predict( df_nuevo_pasajero )

print( y_pred_yo )

[0]
